In [ ]:
import shutil, os

import numpy as np
import torch
from joblib import dump, load

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
features=torch.load('/media/dario/T7_Touch/SC22_Data/features_attentional_maps_and_names/feat.pth')
file_names=torch.load('/media/dario/T7_Touch/SC22_Data/features_attentional_maps_and_names/file_name.pth')

In [ ]:
features.shape

In [ ]:
file_names.shape

In [ ]:
clusters=np.load('/media/dario/T7_Touch/SC22_Data/Clusters/clusters.npy', allow_pickle=True)
dim_red_model=load('/media/dario/T7_Touch/SC22_Data/Clusters/dim_red_model')
scale_model=load('/media/dario/T7_Touch/SC22_Data/Clusters/scale_model')

In [ ]:
vectors=clusters[()]['x']
labels=clusters[()]['y']
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(vectors, labels)

In [ ]:
def get_spectrogram_file_name(fname):
    aux=fname.cpu().detach().numpy().astype(int)
    aux1=str(aux).replace(" ", "")[1:-5]
    aux1=aux1[:8]+'T'+aux1[8:]
    aux1=aux1[:15]+'-'+aux1[15:]+'_Rec_'

    aux2=str(aux).replace(" ", "")[-5:-1].lstrip('0')
    if len(aux2)==0:
        aux2='0'

    output=aux1+aux2+'.jpg'
    return output

In [ ]:
TEXT_FILE_NAME = 'README.txt'
FEATURES_FILE_NAME = 'FEATURES.pth'
DATASET_DIR = 'Labeled_Spectrogram_Images'

PATH = '/media/dario/T7_Touch/SC22_Data/'
IMGS_PATH = '/media/dario/T7_Touch/SC22_Data/Morton_Spectrograms/IMG/'
if not os.path.isdir(os.path.join(PATH,DATASET_DIR)):
    os.makedirs(os.path.join(PATH,DATASET_DIR))
    
f = open(os.path.join(PATH,DATASET_DIR,TEXT_FILE_NAME), 'w')
LINE='Sample num\t' + 'File name\t' + ' Class num'            
f.write(LINE)

    
    


count = 0
features_acum = []
for (feat, fname) in zip(features, file_names):
    file_name=get_spectrogram_file_name(fname=fname)
    print(file_name)

    file_path = os.path.join(IMGS_PATH,file_name)
    feat=np.expand_dims(feat, axis=0)
    r_feat=dim_red_model.transform(scale_model.transform(feat))
    k = neigh.predict(r_feat)[0]
    if k == -1:
        DEST = os.path.join(PATH,DATASET_DIR,'outliers')
        if not os.path.isdir(DEST):
            os.makedirs(DEST)
        
        if os.path.exists(file_path):
            shutil.copy(file_path, DEST)
            
    else:
        DEST = os.path.join(PATH,DATASET_DIR,'Class_' + str(k))
        if not os.path.isdir(DEST):
            os.makedirs(DEST)
        
        if os.path.exists(file_path):
            shutil.copy(file_path, DEST)

            
            
            
    LINE='\n' + str(count) + '\t' + file_name + '\t' + str(k)
    features_acum.append(torch.from_numpy(r_feat[0]))
    f.write(LINE)    
        
    count += 1

f.close()
n_vectors=len(features_acum)
n_feat=features_acum[0].shape[0]
features_acum=torch.cat(features_acum)
features_acum=torch.reshape(features_acum, (n_vectors,n_feat))
features_acum.shape
torch.save(features_acum, os.path.join(PATH,DATASET_DIR,FEATURES_FILE_NAME))